In [5]:
import os
import serial
import time
from datetime import datetime
import smtplib
import getpass
from email.message import EmailMessage
import sys
import socket
socket.getaddrinfo('localhost', 587)

[(<AddressFamily.AF_INET6: 23>, 0, 0, '', ('::1', 587, 0, 0)),
 (<AddressFamily.AF_INET: 2>, 0, 0, '', ('127.0.0.1', 587))]

In [12]:
#robot runner code
robot = "4"

folder = "2_14_22_1"

cycle_time = 15

directory = "D:images\\robot" + robot + "\\" + folder

if not os.path.exists(directory):
    os.makedirs(directory)

com_port = "COM3" #COM3 for robot4

KILLCODE = "2048" #THIS MUST MATCH THE ARDUINO SCRIPT AND MUST BE A STRING, DO NOT CHANGE

#open up serial port with arduino
#arduino = serial.Serial(com_port, 9600, timeout=.1)

In [13]:
def write_read(x):
    arduino.write(bytes(x, 'utf-8'))
    arduino.flushOutput()
    time.sleep(1)
    data = arduino.readline()
    return data

# method prompts the user to enter a value labeled with the variable_name, 
# checks that it's a vlid input inclusively between the upper and lower bound
# and then sends the value to arduino
# it then prints the value returned by arduino as verification that the response is working
def get_user_input(variable_name, lower_bound, upper_bound):
    notValid = True
    while notValid:
        # request user input for the number of shelves
        num = input("Enter a number for %s (must be between %d and %d): "%(variable_name, lower_bound, upper_bound)) # Taking input from user
        numInt = int(num)
        # check if the number entered is valid for the # shelves
        if(numInt >= lower_bound and numInt <= upper_bound):
            notValid = False
        else:
            print("Invalid number: enter an integer between %d and %d"%(lower_bound, upper_bound))
    value = write_read(num)
    print("\tArduino set %s to: %s"%(variable_name, value.decode('utf-8')))
    return numInt

#FUNCTION DECLARATIONS
def password_return():
    while True:
        password1 = getpass.getpass("Please enter google password.")       
        password2 = getpass.getpass("Please re-enter google password.")
        if (password1 == password2):
            break
        else:
            print("passwords did not match.")
    return password1

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith("."):
            yield f

def send_email(password, subject, body):
    fromaddr = "robotmessenger810@gmail.com"
    toaddrs  = "iwtwb8@gmail.com"
    username = "robotmessenger810@gmail.com"
    server = smtplib.SMTP("smtp.gmail.com:587")
    
    # Create the container email message.
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = "robotmessenger810@gmail.com"
    msg['To'] = "iwtwb8@gmail.com"
    msg.set_content(body)
    
    server.starttls()
    server.login(username, password)
    
    server.send_message(msg)
    server.quit()


In [ ]:
# read number of shelves and set expected images per cycle
# num_shelves = int(directory.split("_")[-1])
# images_per_cycle = num_boxes * num_shelves
try:
    #open up serial port with arduino
    arduino = serial.Serial(com_port, 9600, timeout=.3)
    
    # sets the number of shelves in arduino
    num_shelves = get_user_input("Shelves", 1, 4)
    print(num_shelves)
    images_per_cycle = num_shelves * 7
    print(images_per_cycle)
    
    # sets the daylight cycle length in arduino
    day_length = get_user_input("Daylight Cycle Length (enter 25 for constant light)", 0, 25)
    
    # sets the current hour in the daylight cycle
    start_hour = get_user_input("Current hour in cycle", 0, 24)
        
    #return number of files in save directory. Typically this will be 0, unless the robot has been restarted
    start_num_files = len(list(listdir_nohidden(directory)))

    #get current time
    now = datetime.now()

    #kill is a variable that increments each cylce where no images are observed in the save directory. If it reaches 2, 
    #the robot is killed. It is reset to 0 if images are aquired again in the cycle after no images are observed (hard to imagine
    #when this scenario would occur, but it is possible)
    kill = 0

    #setup email server
    #return the password from user input
    pw = password_return()

    #send email after passwored has been input
    send_email(pw, "UPDATE", "Robot" + robot + " login successful.")

    #not sure if this is necessary. 
    arduino.flushOutput()

    #check flycap is all good
    print("Be sure to have FlyCap up and running")
    isReady = False
    
    while not isReady:
        num = input("If FlyCap is running and you're ready, enter 'y' to begin")
        if(num == "y"):
            data = write_read("1")
            isReady = True
    while True:
        isReady = True
        print(arduino.readline())
        #run this while the robot is running
        start = time.time() #this gets the start time of the cycle
        #while(int(time.time() - start) < (60 * 17)):
        while True:
            data = arduino.readline() #if the arduino signals, it means it has returned home
            if data:
                start = time.time()
                arduino.flushInput() #flush input buffer
                current_num_files = len(list(listdir_nohidden(directory))) #get number of saved images
                delta = current_num_files - start_num_files #calculate difference in number of saved images
                start_num_files = current_num_files #reset initial number of saved images for next cycle
                print("current num images: " + str(current_num_files)) #print update
                print("delta: " + str(delta)) #print update 
                
                #this handles all aberrant cases where an unexpected number of images have been saved during a cycle
                if(delta != images_per_cycle): 
                    #this is the case no images were taken
                    if(delta == 0):
                        msg = "ALERT: no images taken this cycle"
                        print(msg)
                        try:
                            send_email(pw, "ALERT", msg)
                        except:
                            print("could not send email")
                        kill = kill + 1
                        
                        #on the second cycle of no-images taken, kill robot.
                        if(kill == 2):
                            msg =  "ALERT: no images taken this cycle. Sending kill signal."
                            print(msg)
                            send_email(pw, "ALERT", msg)
                            
                            #kill sequence
                            arduino.write(1)
                            arduino.flushOutput()
                            arduino.close()
                            sys.exit()
                    
                    #this is the case a non-zero but unexpected number of images were taken. robot still runs.
                    #empirically we know this usually happens due to sluggish saving of the images from the camera.
                    #still haven't figured out exactly what cases the slow saving.
                    else:
                        msg = "ALERT: aberrant number of images taken. Expected: " + str(images_per_cycle) + ". Observed: " + str(delta) + ". Robot will continue to run."
                        print(msg)
                        try:
                            send_email(pw, "ALERT", msg)
                        except:
                            print("could not send email")
                #this is the case an expected number of images were taken. reset kill count and leave loop
                else:
                    kill = 0
                    break
                    
            #testing putting this if statement inside the while loop
            #this block should be entered when the cycle has taken longer than 17 minutes. If so, we kill immediately.
            #It really shouldn't take longer than 3 or 4 minutes. Not sure why I chose 17. 
            if(int(time.time() - start) > (60 * (cycle_time + 2))):
                msg = "ALERT: robot not coming home. Sending kill signal."
                print(msg)
                try:
                    send_email(pw, "ALERT", msg)
                except:
                    print("could not send email") 
                #kill sequence
                write_read(KILLCODE)
                arduino.close()
                sys.exit()

        #get current time. If it is between ~12:42 and ~12:57, send email update
        current_time = datetime.now()
        current_time = current_time.strftime("%H:%M")
        current_time_military = int(str(current_time).split(":")[0]) + int(str(current_time).split(":")[1])/60.

        if(current_time_military >= 13.75 and current_time_military <= 14):
            msg = "UPDATE: Robot" + robot + " is alive and well"
            print(msg)
            try:
                send_email(pw, msg, msg)
            except:
                print("could not send email")
except:
    print("made it to the except statement")
    msg = "ALERT: Run has ended. Sending kill signal."
    print(msg)
    write_read(KILLCODE)
    arduino.close()
    sys.exit()
    try:
        send_email(pw, "ALERT", msg)
    except:
        print("could not send email") 


Enter a number for Shelves (must be between 1 and 4): 1
	Arduino set Shelves to: 1
1
7
Enter a number for Daylight Cycle Length (enter 25 for constant light) (must be between 0 and 25): 25
	Arduino set Daylight Cycle Length (enter 25 for constant light) to: 25
Enter a number for Current hour in cycle (must be between 0 and 24): 9
	Arduino set Current hour in cycle to: 9
Please enter google password.········
Please re-enter google password.········
Be sure to have FlyCap up and running
If FlyCap is running and you're ready, enter 'y' to beginy
b''
current num images: 7
delta: 7
b''
current num images: 14
delta: 7
b''
current num images: 21
delta: 7
b''
current num images: 28
delta: 7
b''
current num images: 35
delta: 7
b''
current num images: 42
delta: 7
b''
current num images: 49
delta: 7
b''
current num images: 56
delta: 7
b''
current num images: 63
delta: 7
b''
current num images: 70
delta: 7
b''
current num images: 77
delta: 7
b''
current num images: 84
delta: 7
b''
current num imag